In [2]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import wave
import numpy as np
from scipy.io import wavfile

In [22]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
import wave
import time


import librosa
from librosa import pyin, note_to_hz
import numpy as np
from threading import Thread
from scipy.stats import skew
import aubio

def analyze_voice(input_data, sample_rate):
    # Normalize amplitudes to the range [-1, 1]
    # normalized_data = input_data / np.max(np.abs(input_data))
    frequencies = np.fft.fftfreq(len(input_data), 1.0 / sample_rate)
    fft_values = np.fft.fft(input_data)

    # Find the index corresponding to the maximum amplitude
    max_index = np.argmax(np.abs(fft_values))

    # Extract the corresponding frequency
    mean_frequency = frequencies[max_index]

    print(f"Mean Frequency: {abs(mean_frequency):.2f} Hz")

    # Perform FFT on the normalized amplitudes
    # fft_result = np.fft.fft(input_data)

    # # Calculate the corresponding frequencies
    # frequencies = np.fft.fftfreq(len(fft_result), 1.0 / sample_rate)

    # # Calculate the squared magnitudes of FFT coefficients
    # magnitude_squared = np.abs(fft_result)**2

    # # print("Frequencies:", frequencies)
    # # print("Magnitudes:", magnitude_squared)


    # # Calculate the mean frequency
    # mean_frequency = np.sum(np.abs(frequencies) * magnitude_squared) / np.sum(magnitude_squared)

    return mean_frequency


def listen(threshold=60):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100

    p = pyaudio.PyAudio()

    #wf = wave.open('output.wav', 'wb')

    # Set the wave file parameters
    #wf.setframerate(RATE)
    #wf.setnchannels(CHANNELS)
    #wf.setsampwidth(2)


    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        # output=True,
        frames_per_buffer=CHUNK
    )

    #stream.start_stream()

    # fig, ax = plt.subplots()

    # x = np.arange(0, 2 * CHUNK, 2)
    # line, = ax.plot(x, np.random.rand(CHUNK))
    # ax.set_ylim(-10000, 10000)
    # ax.set_xlim(0, CHUNK)

    # plt.show(block=False)
    freqs = []
    i = 0
    while i < 70:
        data = stream.read(CHUNK)
        # Convert to numpy array
        audio_data = np.frombuffer(data, dtype=np.int16)
        
        mean_freq = np.array(analyze_voice(audio_data, sample_rate=44100))
        #print(f"Mean Frequency: {mean_freq:.2f} Hz")
        freqs.append(mean_freq)
        if i == 69:
            freqs = np.array(freqs)
            freq0, voiced_flag, voiced_probs = pyin(
                y=freqs, 
                fmin=note_to_hz('C2'), 
                fmax=note_to_hz('C7'), 
                sr=44100, 
                frame_length=2048
            )

            # freq0 = freq0[voiced_flag]
            # freq0 = np.array(freq0[freq0 > 0])
            # freq0 /= 1000
            # print(freqs)
            print(freq0)
        # Calculate the amplitude (volume) of the audio chunk
        #amplitude = np.abs(audio_data).mean()
        #print(amplitude)
        # Check for silence
        #if amplitude > threshold:
            #wf.writeframes(data)

        # amps = np.frombuffer(data, np.int16)
        # line.set_ydata(amps)
        # plt.draw()
        # fig.canvas.flush_events()
        i += 1

    stream.stop_stream()

# t1 = Thread(target=listen)
# t2 = Thread(target=extract_features)

# t1.start()
# t2.start()
# t1.join()
# t2.join()


# audio_file = 'output.wav'
# results = specan_python(audio_file)
# print(results)

listen()

Mean Frequency: 12518599.37 Hz
Mean Frequency: -291618.70 Hz
Mean Frequency: 105123.42 Hz
Mean Frequency: -584482.59 Hz
Mean Frequency: 523082.76 Hz
Mean Frequency: -625244.14 Hz
Mean Frequency: 18364.67 Hz
Mean Frequency: 222275.19 Hz
Mean Frequency: 4754.84 Hz
Mean Frequency: -112739.69 Hz
Mean Frequency: -3792694.04 Hz
Mean Frequency: 548594.48 Hz
Mean Frequency: 1210652.87 Hz
Mean Frequency: 3871603.35 Hz
Mean Frequency: 464296.36 Hz
Mean Frequency: -1654932.90 Hz
Mean Frequency: 513824.37 Hz
Mean Frequency: -474572.37 Hz
Mean Frequency: -1942200.80 Hz
Mean Frequency: 4409765.45 Hz
Mean Frequency: 2104068.91 Hz
Mean Frequency: 632779.63 Hz
Mean Frequency: -79191.85 Hz
Mean Frequency: 488935.56 Hz
Mean Frequency: -1114785.83 Hz
Mean Frequency: 20729.91 Hz
Mean Frequency: 415002.78 Hz
Mean Frequency: 1153693.00 Hz
Mean Frequency: 121077.58 Hz
Mean Frequency: -140109.87 Hz
Mean Frequency: 690598.34 Hz
Mean Frequency: -509680.28 Hz
Mean Frequency: 349451.46 Hz
Mean Frequency: -115994.2

In [25]:

def calculate_mean_frequency():
    CHUNK = 1024  # Number of frames per buffer
    FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
    CHANNELS = 1  # Number of audio channels (1 for mono, 2 for stereo)
    RATE = 44100  # Sampling rate
    RECORD_SECONDS = 10  # Duration of recording in seconds

    p = pyaudio.PyAudio()

    stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        frames_per_buffer=CHUNK
    )

    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording complete.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    audio_data = np.frombuffer(b''.join(frames), dtype=np.int16)

    # Compute the Fast Fourier Transform (FFT)
    frequencies = np.fft.fftfreq(len(audio_data), 1.0 / RATE)
    fft_values = np.fft.fft(audio_data)

    # Find the index corresponding to the maximum amplitude
    max_index = np.argmax(np.abs(fft_values))

    # Extract the corresponding frequency
    mean_frequency = frequencies[max_index]

    print(f"Mean Frequency: {abs(mean_frequency):.2f} Hz")

if __name__ == "__main__":
    calculate_mean_frequency()


Recording...
Recording complete.
Mean Frequency: 383.99 Hz
